# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770968878662                   -0.53    9.0    185ms
  2   -2.772151464351       -2.93       -1.32    1.0    113ms
  3   -2.772170381814       -4.72       -2.49    1.0    107ms
  4   -2.772170667862       -6.54       -3.24    1.0    109ms
  5   -2.772170722700       -7.26       -4.11    2.0    128ms
  6   -2.772170722917       -9.66       -4.41    1.0    128ms
  7   -2.772170723011      -10.03       -5.34    1.0    117ms
  8   -2.772170723015      -11.38       -5.92    2.0    140ms
  9   -2.772170723015      -12.96       -6.42    1.0    119ms
 10   -2.772170723015      -13.61       -7.05    1.0    135ms
 11   -2.772170723015   +  -14.10       -7.64    2.0    148ms
 12   -2.772170723015      -13.77       -7.80    1.0    124ms
 13   -2.772170723015   +  -14.57       -8.73    1.0    137ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.773558050442181

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770681855296                   -0.53    9.0    183ms
  2   -2.772052743678       -2.86       -1.31    1.0    104ms
  3   -2.772083137371       -4.52       -2.60    1.0    116ms
  4   -2.772083402294       -6.58       -3.90    1.0    124ms
  5   -2.772083417329       -7.82       -4.24    2.0    149ms
  6   -2.772083417791       -9.34       -5.50    1.0    118ms
  7   -2.772083417810      -10.73       -5.58    2.0    131ms
  8   -2.772083417811      -12.05       -6.67    1.0    143ms
  9   -2.772083417811      -13.60       -6.68    2.0    140ms
 10   -2.772083417811      -13.69       -7.54    1.0    425ms
 11   -2.772083417811   +  -13.77       -8.21    1.0    377ms

Polarizability via ForwardDiff:       1.772534970471691
Polarizability via finite difference: 1.773558050442181
